# Category2.유동인구많은곳_전처리

### 목차
- (1) 데이터 전처리
    - 1 정류장 데이터
    - 2 혼잡 빈도 강도, 혼잡 시간 강도 
    - 3 교통량
    - 4 버스승하차
    - 5 유동인구  



- (2) 군집화
    - 1 결측값 대체
    - 2 K-means 모델링
        - 2-1 데이터 표준화
        - 2-2 적절한 k값 찾기
        - 2-3 K-means train
        - 2-4 t-SNE 
    - 3 분석 결과
        - 3-1 군집별 점수 부여
        - 3-2 군집 내 2차 분류
        

In [1]:
from geoband.API import *
GetCompasData('SBJ_2102_002', '1', '1.수원시_버스정류장.csv')
GetCompasData('SBJ_2102_002', '2', '2.수원시_버스정류장별_승하차이력(1).csv')
GetCompasData('SBJ_2102_002', '3', '3.수원시_버스정류장별_승하차이력(2).csv')
GetCompasData('SBJ_2102_002', '4', '4.수원시_버스정류장별_승하차이력(3).csv')
GetCompasData('SBJ_2102_002', '5', '5.수원시_버스정류장별_승하차이력(4).csv')
GetCompasData('SBJ_2102_002', '6', '6.수원시_버스정류장별_승하차이력(5).csv')
GetCompasData('SBJ_2102_002', '16', '16.수원시_요일별_유동인구(2020).csv')
GetCompasData('SBJ_2102_002', '24', '24.수원시_평일_일별_혼잡빈도강도_LV6.csv')
GetCompasData('SBJ_2102_002', '25', '25.수원시_평일_일별_혼잡시간강도_LV6.csv')


## (1) 데이터 전처리

In [2]:
# 필요한 라이브러리 호출
import pandas as pd
import warnings 
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, LineString, Point
warnings.filterwarnings(action='ignore')

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### 1. 정류장 데이터

In [3]:
bus_station= pd.read_csv('1.수원시_버스정류장.csv')
bus_station

,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat
0,201000313,권선구청,"시내,마을",일반,경기도 수원시 서둔동,1.0,1.0,1.0,NaN,NaN,NaN,NaN,126.970983,37.256983
1,201000344,중보들공원,"시내,마을",일반,경기도 수원시 평동,1.0,NaN,NaN,NaN,NaN,NaN,NaN,126.991029,37.244605
2,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,NaN,1.0,NaN,NaN,NaN,1.0,NaN,127.016000,37.251091
3,201000379,신미주아파트,시내,일반,경기도 수원시 금곡동,NaN,1.0,1.0,NaN,NaN,NaN,NaN,126.953845,37.268806
4,201000055,농수산물시장,시내,일반,경기도 수원시 권선1동,NaN,1.0,NaN,1.0,NaN,NaN,NaN,127.029815,37.256755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174,201000266,구운초등학교,시내,일반,경기도 수원시 구운동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.978712,37.281431
1175,228000723,경희대정문,"시내,마을",일반,경기도 수원시 영통1동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.077907,37.247475
1176,233001574,수영리검사소,"시내,마을",일반,경기도 수원시 평동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.968503,37.232512
1177,201000123,자혜학교,시내,일반,경기도 수원시 서둔동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.977880,37.265151


In [4]:
#BIS 설치 안된 정류장 제외하고 분석

bus_station.drop(bus_station[bus_station['BIS설치여부']!=1].index, inplace=True)
bus_station.reset_index(inplace=True, drop=True)
bus_station

,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat
0,201000313,권선구청,"시내,마을",일반,경기도 수원시 서둔동,1.0,1.0,1.0,NaN,NaN,NaN,NaN,126.970983,37.256983
1,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,NaN,1.0,NaN,NaN,NaN,1.0,NaN,127.016000,37.251091
2,201000379,신미주아파트,시내,일반,경기도 수원시 금곡동,NaN,1.0,1.0,NaN,NaN,NaN,NaN,126.953845,37.268806
3,201000055,농수산물시장,시내,일반,경기도 수원시 권선1동,NaN,1.0,NaN,1.0,NaN,NaN,NaN,127.029815,37.256755
4,201000268,곡반중학교,시내,일반,경기도 수원시 곡선동,1.0,1.0,1.0,NaN,NaN,NaN,NaN,127.034254,37.246081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,202000249,화서오거리,시내,일반,경기도 수원시 화서1동,NaN,1.0,NaN,NaN,1.0,NaN,NaN,126.999697,37.278865
924,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,"시내,마을",일반,경기도 수원시 화서2동,NaN,1.0,NaN,1.0,NaN,NaN,NaN,126.995823,37.284316
925,202000246,엘지아파트.금강아파트,시내,일반,경기도 수원시 화서2동,1.0,1.0,NaN,NaN,1.0,NaN,NaN,126.981714,37.287994
926,202000239,꽃뫼신동아아파트,"시내,마을",일반,경기도 수원시 화서2동,NaN,1.0,NaN,1.0,NaN,NaN,NaN,126.986076,37.286663


In [5]:
bus_station['geometry'] = bus_station.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
bus_station['geometry'] = gpd.GeoDataFrame(bus_station['geometry'], geometry='geometry') # 시각화를 위해 좌표묶기
bus_station.head()

,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,geometry
0,201000313,권선구청,"시내,마을",일반,경기도 수원시 서둔동,1.0,1.0,1.0,NaN,NaN,NaN,NaN,126.970983,37.256983,POINT (126.9709828 37.25698318)
1,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,NaN,1.0,NaN,NaN,NaN,1.0,NaN,127.016000,37.251091,POINT (127.0160003 37.25109066)
2,201000379,신미주아파트,시내,일반,경기도 수원시 금곡동,NaN,1.0,1.0,NaN,NaN,NaN,NaN,126.953845,37.268806,POINT (126.9538449 37.26880555)
3,201000055,농수산물시장,시내,일반,경기도 수원시 권선1동,NaN,1.0,NaN,1.0,NaN,NaN,NaN,127.029815,37.256755,POINT (127.029815 37.25675503)
4,201000268,곡반중학교,시내,일반,경기도 수원시 곡선동,1.0,1.0,1.0,NaN,NaN,NaN,NaN,127.034254,37.246081,POINT (127.034254 37.24608082)


In [6]:
# 동만을 추출하여 새로운 컬럼 생성
bus_station['동']=bus_station['위치(동)'].str.split(" ").str[2].astype(str)
bus_station

,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,geometry,동
0,201000313,권선구청,"시내,마을",일반,경기도 수원시 서둔동,1.0,1.0,1.0,NaN,NaN,NaN,NaN,126.970983,37.256983,POINT (126.9709828 37.25698318),서둔동
1,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,NaN,1.0,NaN,NaN,NaN,1.0,NaN,127.016000,37.251091,POINT (127.0160003 37.25109066),세류2동
2,201000379,신미주아파트,시내,일반,경기도 수원시 금곡동,NaN,1.0,1.0,NaN,NaN,NaN,NaN,126.953845,37.268806,POINT (126.9538449 37.26880555),금곡동
3,201000055,농수산물시장,시내,일반,경기도 수원시 권선1동,NaN,1.0,NaN,1.0,NaN,NaN,NaN,127.029815,37.256755,POINT (127.029815 37.25675503),권선1동
4,201000268,곡반중학교,시내,일반,경기도 수원시 곡선동,1.0,1.0,1.0,NaN,NaN,NaN,NaN,127.034254,37.246081,POINT (127.034254 37.24608082),곡선동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,202000249,화서오거리,시내,일반,경기도 수원시 화서1동,NaN,1.0,NaN,NaN,1.0,NaN,NaN,126.999697,37.278865,POINT (126.9996969 37.27886499),화서1동
924,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,"시내,마을",일반,경기도 수원시 화서2동,NaN,1.0,NaN,1.0,NaN,NaN,NaN,126.995823,37.284316,POINT (126.9958225 37.28431556),화서2동
925,202000246,엘지아파트.금강아파트,시내,일반,경기도 수원시 화서2동,1.0,1.0,NaN,NaN,1.0,NaN,NaN,126.981714,37.287994,POINT (126.9817138 37.28799446),화서2동
926,202000239,꽃뫼신동아아파트,"시내,마을",일반,경기도 수원시 화서2동,NaN,1.0,NaN,1.0,NaN,NaN,NaN,126.986076,37.286663,POINT (126.9860763 37.28666265),화서2동


In [7]:
# 사용될 칼럼만을 뽑아 새로운 dataframe생성
bus_station=bus_station.loc[:,['정류장ID','정류장명','동','geometry','lon','lat']]
bus_station

,정류장ID,정류장명,동,geometry,lon,lat
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983
1,201000345,남수원초등학교,세류2동,POINT (127.0160003 37.25109066),127.016000,37.251091
2,201000379,신미주아파트,금곡동,POINT (126.9538449 37.26880555),126.953845,37.268806
3,201000055,농수산물시장,권선1동,POINT (127.029815 37.25675503),127.029815,37.256755
4,201000268,곡반중학교,곡선동,POINT (127.034254 37.24608082),127.034254,37.246081
...,...,...,...,...,...,...
923,202000249,화서오거리,화서1동,POINT (126.9996969 37.27886499),126.999697,37.278865
924,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,화서2동,POINT (126.9958225 37.28431556),126.995823,37.284316
925,202000246,엘지아파트.금강아파트,화서2동,POINT (126.9817138 37.28799446),126.981714,37.287994
926,202000239,꽃뫼신동아아파트,화서2동,POINT (126.9860763 37.28666265),126.986076,37.286663


[https://stat.suwon.go.kr/stat/stats/statsView.do?categorySeqNo=14] 출처

In [8]:
resi_population = pd.read_excel('data/수원시 인구통계(구,동,성별).xls')
resi_population=resi_population.loc[:,['동','인구_계']]
resi_population

,동,인구_계
0,파장동,23573
1,율천동,43017
2,정자1동,29779
3,정자2동,30060
4,정자3동,42707
5,영화동,20355
6,송죽동,19667
7,조원1동,28763
8,조원2동,19103
9,연무동,16300


In [9]:
# 거주인구 데이터에서 동별로 정류장데이터와 merge
data=pd.merge(bus_station,resi_population,how='left', on='동')
data

,정류장ID,정류장명,동,geometry,lon,lat,인구_계
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0
1,201000345,남수원초등학교,세류2동,POINT (127.0160003 37.25109066),127.016000,37.251091,23728.0
2,201000379,신미주아파트,금곡동,POINT (126.9538449 37.26880555),126.953845,37.268806,45794.0
3,201000055,농수산물시장,권선1동,POINT (127.029815 37.25675503),127.029815,37.256755,24895.0
4,201000268,곡반중학교,곡선동,POINT (127.034254 37.24608082),127.034254,37.246081,27497.0
...,...,...,...,...,...,...,...
923,202000249,화서오거리,화서1동,POINT (126.9996969 37.27886499),126.999697,37.278865,23411.0
924,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,화서2동,POINT (126.9958225 37.28431556),126.995823,37.284316,26776.0
925,202000246,엘지아파트.금강아파트,화서2동,POINT (126.9817138 37.28799446),126.981714,37.287994,26776.0
926,202000239,꽃뫼신동아아파트,화서2동,POINT (126.9860763 37.28666265),126.986076,37.286663,26776.0


In [10]:
data[data['동']=='광교1동']
data['인구_계'].max()

53158.0

## 2.혼잡빈도강도, 혼잡시간강도

#### 방법: 각 정류소 반경 500m 이내 도로의 혼잡빈도강도의 평균을 해당 정류소의 혼잡빈도강도로 간주
#### data24 혼잡빈도강도 , 혼잡시간강도데이터에는 도로의 좌표 데이터가 없음 
####  ->>미세먼지노출지수에서 전처리한 도로링크+좌표 데이터를 이용하기
## 
### 2-1. 혼잡빈도강도,혼잡시간강도 데이터와 traffic_link(도로링크+좌표) 를 link_id 기준으로 merge

-혼잡빈도강도 전처리

In [11]:
crowded1=pd.read_csv('24.수원시_평일_일별_혼잡빈도강도_LV6.csv')

In [12]:
crowded1.head()

,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,혼잡빈도강도
0,47835381601,101,3.130,수원광명고속도로,경기도,수원시권선구,금곡동,25.93
1,47835381901,101,3.130,수원광명고속도로,경기도,수원시권선구,금곡동,27.30
2,47846654001,101,0.015,수원광명고속도로,경기도,수원시권선구,입북동,55.74
3,47846654101,101,0.036,수원광명고속도로,경기도,수원시권선구,입북동,53.78
4,47846654301,101,0.036,수원광명고속도로,경기도,수원시권선구,입북동,55.70


In [13]:
#혼잡빈도강도의 상세도로망_linkID와 상세도로망 데이터의 link_id를 기준으로 merge하기 
#merge를 위해 상세도로망_linkID 일부 변경
crowded1['상세도로망_LinkID'] = crowded1['상세도로망_LinkID'].astype(str)
crowded1['link_id']=''
for i in range(len(crowded1)):
    crowded1['link_id'][i] = crowded1['상세도로망_LinkID'][i][:-2]

In [14]:
crowded1.head()

,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,혼잡빈도강도,link_id
0,47835381601,101,3.130,수원광명고속도로,경기도,수원시권선구,금곡동,25.93,478353816
1,47835381901,101,3.130,수원광명고속도로,경기도,수원시권선구,금곡동,27.30,478353819
2,47846654001,101,0.015,수원광명고속도로,경기도,수원시권선구,입북동,55.74,478466540
3,47846654101,101,0.036,수원광명고속도로,경기도,수원시권선구,입북동,53.78,478466541
4,47846654301,101,0.036,수원광명고속도로,경기도,수원시권선구,입북동,55.70,478466543


In [15]:
# 미세먼지노출지수에서 전처리한 데이터 이용
traffic_link = pd.read_csv('data/traffic_link.csv')
traffic_link

,link_id,시군구명,읍면동명,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량,상세도로망_LinkID,geometry,lon,lat
0,478353816,수원시권선구,금곡동,11690,8588,424,2678,47835381601,MULTILINESTRING ((126.9499376616103 37.2800703...,126.949943,37.279939
1,478353819,수원시권선구,금곡동,14564,10664,696,3203,47835381901,MULTILINESTRING ((126.9497154246591 37.2519931...,126.949658,37.252413
2,478466540,수원시권선구,입북동,17031,12607,604,3820,47846654001,MULTILINESTRING ((126.9485860517042 37.2895445...,126.948599,37.289476
3,478466541,수원시권선구,입북동,18025,13367,612,4046,47846654101,MULTILINESTRING ((126.9485113944438 37.2898699...,126.948549,37.289707
4,478466543,수원시권선구,입북동,19989,14868,762,4359,47846654301,MULTILINESTRING ((126.9487574825045 37.2895592...,126.948721,37.289721
...,...,...,...,...,...,...,...,...,...,...,...
11663,478442493,수원시장안구,파장동,16417,12142,446,3830,47844249301,MULTILINESTRING ((126.9939178199916 37.3173855...,126.993797,37.317483
11664,478442498,수원시장안구,파장동,3746,2831,104,812,47844249801,MULTILINESTRING ((126.9911422681942 37.3184533...,126.991358,37.318311
11665,478442497,수원시장안구,파장동,19727,14766,907,4054,47844249701,MULTILINESTRING ((126.9913317103872 37.3205848...,126.991289,37.320706
11666,478442264,수원시장안구,파장동,26583,19942,1047,5594,47844226401,MULTILINESTRING ((126.988004644842 37.32577995...,126.988119,37.325603


In [16]:
#필요한 컬럼만 추출
traffic_link = traffic_link[['link_id','lon','lat']]

In [17]:
#merge를 위해 같은 타입으로 변환
crowded1['link_id'] = crowded1['link_id'].astype(str)
traffic_link['link_id'] = traffic_link['link_id'].astype(str)

In [18]:
crowded1 = pd.merge(crowded1, traffic_link,how='left',on=['link_id'])
crowded1 = crowded1[['상세도로망_LinkID','혼잡빈도강도','link_id','lon','lat']]
crowded1.head()

,상세도로망_LinkID,혼잡빈도강도,link_id,lon,lat
0,47835381601,25.93,478353816,126.949943,37.279939
1,47835381901,27.30,478353819,126.949658,37.252413
2,47846654001,55.74,478466540,126.948599,37.289476
3,47846654101,53.78,478466541,126.948549,37.289707
4,47846654301,55.70,478466543,126.948721,37.289721


In [19]:
crowded_sum1=crowded1[['link_id','혼잡빈도강도']].groupby(['link_id'], as_index=False).sum()
crowded_sum1

,link_id,혼잡빈도강도
0,478300330,367.56
1,478300581,388.70
2,478310046,392.34
3,478310049,392.36
4,478310050,393.58
...,...,...
6180,571492408,25.46
6181,571492534,58.36
6182,571492535,45.77
6183,571492538,73.80


In [20]:
crowded_freq = pd.merge(crowded_sum1,crowded1[['link_id','lon','lat']],how='left',on='link_id')
crowded_freq=crowded_freq.drop_duplicates()
crowded_freq.reset_index(drop=True,inplace=True)
crowded_freq

,link_id,혼잡빈도강도,lon,lat
0,478300330,367.56,126.963462,37.245513
1,478300581,388.70,126.969336,37.247611
2,478310046,392.34,126.982018,37.251183
3,478310049,392.36,126.982440,37.251304
4,478310050,393.58,126.980647,37.250791
...,...,...,...,...
6180,571492408,25.46,127.061782,37.270343
6181,571492534,58.36,127.023915,37.313558
6182,571492535,45.77,127.021678,37.313000
6183,571492538,73.80,127.022219,37.312967


-혼잡시간강도 전처리

In [21]:
crowded2=pd.read_csv('25.수원시_평일_일별_혼잡시간강도_LV6.csv')
crowded2.head()

,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,혼잡시간강도
0,47835381601,101,3.130,수원광명고속도로,경기도,수원시권선구,금곡동,31.83
1,47835381901,101,3.130,수원광명고속도로,경기도,수원시권선구,금곡동,33.87
2,47846654001,101,0.015,수원광명고속도로,경기도,수원시권선구,입북동,66.87
3,47846654101,101,0.036,수원광명고속도로,경기도,수원시권선구,입북동,64.91
4,47846654301,101,0.036,수원광명고속도로,경기도,수원시권선구,입북동,75.29


In [22]:
#혼잡시간강도의 상세도로망_linkID와 상세도로망 데이터의 link_id를 기준으로 merge하기 
#merge를 위해 상세도로망_linkID 일부 변경
crowded2['상세도로망_LinkID'] = crowded2['상세도로망_LinkID'].astype(str)
crowded2['link_id']=''
for i in range(len(crowded2)):
    crowded2['link_id'][i] = crowded2['상세도로망_LinkID'][i][:-2]
crowded2

,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,혼잡시간강도,link_id
0,47835381601,101,3.130,수원광명고속도로,경기도,수원시권선구,금곡동,31.83,478353816
1,47835381901,101,3.130,수원광명고속도로,경기도,수원시권선구,금곡동,33.87,478353819
2,47846654001,101,0.015,수원광명고속도로,경기도,수원시권선구,입북동,66.87,478466540
3,47846654101,101,0.036,수원광명고속도로,경기도,수원시권선구,입북동,64.91,478466541
4,47846654301,101,0.036,수원광명고속도로,경기도,수원시권선구,입북동,75.29,478466543
...,...,...,...,...,...,...,...,...,...
10850,47844582002,107,0.347,수성로,경기도,수원시팔달구,화서2동,45.38,478445820
10851,47844197402,107,0.587,정자천로,경기도,수원시팔달구,화서2동,6.49,478441974
10852,47844197401,107,0.587,정자천로,경기도,수원시팔달구,화서2동,13.94,478441974
10853,47844917502,107,0.604,NaN,경기도,수원시팔달구,화서2동,26.24,478449175


In [23]:
crowded2['link_id'] = crowded2['link_id'].astype(str)

In [24]:
crowded2 = pd.merge(crowded2[['link_id','혼잡시간강도','상세도로망_LinkID']], traffic_link,how='left',on=['link_id'])
crowded2 = crowded2[['상세도로망_LinkID','혼잡시간강도','link_id','lon','lat']]
crowded2.head()

,상세도로망_LinkID,혼잡시간강도,link_id,lon,lat
0,47835381601,31.83,478353816,126.949943,37.279939
1,47835381901,33.87,478353819,126.949658,37.252413
2,47846654001,66.87,478466540,126.948599,37.289476
3,47846654101,64.91,478466541,126.948549,37.289707
4,47846654301,75.29,478466543,126.948721,37.289721


In [25]:
crowded_sum2=crowded2[['link_id','혼잡시간강도']].groupby(['link_id'], as_index=False).sum()
crowded_sum2

,link_id,혼잡시간강도
0,478300330,393.22
1,478300581,394.26
2,478310046,397.32
3,478310049,396.86
4,478310050,397.74
...,...,...
6180,571492408,59.97
6181,571492534,65.20
6182,571492535,53.28
6183,571492538,78.78


In [26]:
crowded_time = pd.merge(crowded_sum2,crowded2[['link_id','lon','lat']],how='left',on='link_id')
crowded_time=crowded_time.drop_duplicates()
crowded_time.reset_index(drop=True,inplace=True)
crowded_time

,link_id,혼잡시간강도,lon,lat
0,478300330,393.22,126.963462,37.245513
1,478300581,394.26,126.969336,37.247611
2,478310046,397.32,126.982018,37.251183
3,478310049,396.86,126.982440,37.251304
4,478310050,397.74,126.980647,37.250791
...,...,...,...,...
6180,571492408,59.97,127.061782,37.270343
6181,571492534,65.20,127.023915,37.313558
6182,571492535,53.28,127.021678,37.313000
6183,571492538,78.78,127.022219,37.312967


혼잡빈도강도 + 혼잡시간강도 merge

In [27]:
crowded = pd.merge(crowded_freq,crowded_time[['link_id','혼잡시간강도']],how='left',on='link_id')
crowded.head()

,link_id,혼잡빈도강도,lon,lat,혼잡시간강도
0,478300330,367.56,126.963462,37.245513,393.22
1,478300581,388.70,126.969336,37.247611,394.26
2,478310046,392.34,126.982018,37.251183,397.32
3,478310049,392.36,126.982440,37.251304,396.86
4,478310050,393.58,126.980647,37.250791,397.74


In [28]:
crowded['lon'] = crowded['lon'].astype(float)
crowded['lat'] = crowded['lat'].astype(float)

### 2-2. 버스정류소와 모든 도로간의 거리를 harversine공식을 이해 구한 후, 반경 300m 이내인 도로만 추출

#### -빠른 계산을 위해 array 형태로 변환

In [29]:
#버스정류소 좌표 array 로 만들기 
bus_array = data[['정류장ID','lon','lat']]
bus_array['정류장ID'] = bus_array['정류장ID'].astype(str)
bus_array = bus_array[['정류장ID','lon','lat']].to_numpy()
bus_array

array([['201000313', 126.9709828, 37.25698318],
       ['201000345', 127.0160003, 37.25109066],
       ['201000379', 126.9538449, 37.26880555],
       ...,
       ['202000246', 126.98171380000001, 37.28799446],
       ['202000239', 126.98607630000001, 37.28666265],
       ['202000238', 126.9850962, 37.28950644]], dtype=object)

In [30]:
#도로 좌표 array로 만들기
road_array = crowded[['link_id','lon','lat']]
road_array['link_id'] = crowded['link_id'].astype(str)
road_array = road_array.to_numpy()
road_array

array([['478300330', 126.96346226486608, 37.24551252856579],
       ['478300581', 126.96933640270609, 37.247610754658275],
       ['478310046', 126.98201785069216, 37.25118274591618],
       ...,
       ['571492535', 127.02167768460446, 37.31299976687368],
       ['571492538', 127.02221909790876, 37.312966502174156],
       ['571492539', 127.02055938003099, 37.3124987306041]], dtype=object)

In [31]:
print(bus_array.shape[0],road_array.shape[0])

928 6185


In [32]:
#하버사인 거리 구하기
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)


In [33]:
#반경 150m 이내인 도로만 리스트에 추가 (도로 여러개일 수 있기때문에 중첩리스트 형식으로 추가)
index_150m = []
for i in range(bus_array.shape[0]):
    each_road=[]
    for j in range(road_array.shape[0]):
        h_dis = GeoUtil.get_harversion_distance(bus_array[i,1], bus_array[i,2], road_array[j,1],road_array[j,2])
        if h_dis<=0.15:  
            each_road.append(road_array[j,0])
            
    index_150m.append(each_road) #index에 해당하는 link_id를 리스트에 append

In [34]:
#중복되는 도로 삭제
for i in range(len(index_150m)):
    index_150m[i] = list(set(index_150m[i]))

In [35]:
len(index_150m)

928

In [36]:
index_150m

[['478449676',
  '478466694',
  '478466704',
  '478448892',
  '478456792',
  '478449058',
  '478449679',
  '478466693',
  '478456776'],
 ['571346114',
  '571346248',
  '571346250',
  '571346249',
  '571346209',
  '571346212',
  '571346133',
  '571372086'],
 ['478442332',
  '478448434',
  '478450906',
  '478444899',
  '478444813',
  '478444842',
  '478450905',
  '478446203'],
 ['571410962', '571410957', '571410946'],
 ['571369580',
  '571369578',
  '571344582',
  '571369566',
  '571365010',
  '571369570',
  '571369568',
  '571341433'],
 ['478458795', '478458799', '478458968', '478462766'],
 ['478448976', '478449298', '478452234', '478448962'],
 ['478445845', '478451156', '478451157', '478448122', '478448125'],
 ['478459256', '478459254', '478444829'],
 ['478462878',
  '478459452',
  '478459256',
  '478444829',
  '478459454',
  '478444850',
  '478462220',
  '478462218',
  '478459254'],
 ['478444945', '478462852', '478448422', '478448447'],
 ['478353789', '478342313', '478342315', '478342

In [37]:
#위에서 구한 도로 리스트를 컬럼으로 생성
data['link_id'] = index_150m
data

,정류장ID,정류장명,동,geometry,lon,lat,인구_계,link_id
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,"[478449676, 478466694, 478466704, 478448892, 4..."
1,201000345,남수원초등학교,세류2동,POINT (127.0160003 37.25109066),127.016000,37.251091,23728.0,"[571346114, 571346248, 571346250, 571346249, 5..."
2,201000379,신미주아파트,금곡동,POINT (126.9538449 37.26880555),126.953845,37.268806,45794.0,"[478442332, 478448434, 478450906, 478444899, 4..."
3,201000055,농수산물시장,권선1동,POINT (127.029815 37.25675503),127.029815,37.256755,24895.0,"[571410962, 571410957, 571410946]"
4,201000268,곡반중학교,곡선동,POINT (127.034254 37.24608082),127.034254,37.246081,27497.0,"[571369580, 571369578, 571344582, 571369566, 5..."
...,...,...,...,...,...,...,...,...
923,202000249,화서오거리,화서1동,POINT (126.9996969 37.27886499),126.999697,37.278865,23411.0,"[571410304, 571410708, 571410650, 571410699]"
924,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,화서2동,POINT (126.9958225 37.28431556),126.995823,37.284316,26776.0,"[478450414, 478444723, 478446540, 478410672, 4..."
925,202000246,엘지아파트.금강아파트,화서2동,POINT (126.9817138 37.28799446),126.981714,37.287994,26776.0,"[478448044, 478448059, 478461542, 478463960, 4..."
926,202000239,꽃뫼신동아아파트,화서2동,POINT (126.9860763 37.28666265),126.986076,37.286663,26776.0,"[478456666, 478444262, 478444268, 478449768, 4..."


#### -정류소별로 5000m이내인 도로 여러개 있기 때문에 모두 행으로 펼쳐서 데이터프레임 만들기

In [38]:
for i in range(len(data)):
    data['link_id'][i] = ' '.join(data['link_id'][i]) 
data

,정류장ID,정류장명,동,geometry,lon,lat,인구_계,link_id
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478449676 478466694 478466704 478448892 478456...
1,201000345,남수원초등학교,세류2동,POINT (127.0160003 37.25109066),127.016000,37.251091,23728.0,571346114 571346248 571346250 571346249 571346...
2,201000379,신미주아파트,금곡동,POINT (126.9538449 37.26880555),126.953845,37.268806,45794.0,478442332 478448434 478450906 478444899 478444...
3,201000055,농수산물시장,권선1동,POINT (127.029815 37.25675503),127.029815,37.256755,24895.0,571410962 571410957 571410946
4,201000268,곡반중학교,곡선동,POINT (127.034254 37.24608082),127.034254,37.246081,27497.0,571369580 571369578 571344582 571369566 571365...
...,...,...,...,...,...,...,...,...
923,202000249,화서오거리,화서1동,POINT (126.9996969 37.27886499),126.999697,37.278865,23411.0,571410304 571410708 571410650 571410699
924,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,화서2동,POINT (126.9958225 37.28431556),126.995823,37.284316,26776.0,478450414 478444723 478446540 478410672 478457...
925,202000246,엘지아파트.금강아파트,화서2동,POINT (126.9817138 37.28799446),126.981714,37.287994,26776.0,478448044 478448059 478461542 478463960 478448...
926,202000239,꽃뫼신동아아파트,화서2동,POINT (126.9860763 37.28666265),126.986076,37.286663,26776.0,478456666 478444262 478444268 478449768 478461...


In [39]:
# 공백 단위로 도로 분할
#traffic_link['link_id'] = traffic_link['link_id'].astype(str)
split =data.link_id.str.split(' ')
split

0      [478449676, 478466694, 478466704, 478448892, 4...
1      [571346114, 571346248, 571346250, 571346249, 5...
2      [478442332, 478448434, 478450906, 478444899, 4...
3                      [571410962, 571410957, 571410946]
4      [571369580, 571369578, 571344582, 571369566, 5...
                             ...                        
923         [571410304, 571410708, 571410650, 571410699]
924    [478450414, 478444723, 478446540, 478410672, 4...
925    [478448044, 478448059, 478461542, 478463960, 4...
926    [478456666, 478444262, 478444268, 478449768, 4...
927    [478446589, 478401578, 478446862, 478451215, 4...
Name: link_id, Length: 928, dtype: object

In [40]:
split = split.apply(lambda x: pd.Series(x))
split

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,478449676,478466694,478466704,478448892,478456792,478449058,478449679,478466693,478456776,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,571346114,571346248,571346250,571346249,571346209,571346212,571346133,571372086,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,478442332,478448434,478450906,478444899,478444813,478444842,478450905,478446203,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,571410962,571410957,571410946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,571369580,571369578,571344582,571369566,571365010,571369570,571369568,571341433,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,571410304,571410708,571410650,571410699,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
924,478450414,478444723,478446540,478410672,478457272,478450416,478444722,478452804,478457274,478451676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
925,478448044,478448059,478461542,478463960,478448065,478448060,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
926,478456666,478444262,478444268,478449768,478461480,478461488,478444269,478444264,478444276,478461490,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#도로 모두 행으로 변환
split.stack().reset_index(level=1, drop=True)

0      478449676
0      478466694
0      478466704
0      478448892
0      478456792
         ...    
927    478442390
927    478446861
927    478446254
927    478444165
927    478446255
Length: 8082, dtype: object

In [42]:
#행으로 변환한 도로 데이터프레임으로 만들기
split = split.stack().reset_index(level=1, drop=True).to_frame('link_id')
split

,link_id
0,478449676
0,478466694
0,478466704
0,478448892
0,478456792
...,...
927,478442390
927,478446861
927,478446254
927,478444165


In [43]:
#기존 데이터에 merge 하기
data = data.merge(split, left_index=True, right_index=True, how='left')
data.head()

,정류장ID,정류장명,동,geometry,lon,lat,인구_계,link_id_x,link_id_y
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478449676 478466694 478466704 478448892 478456...,478449676
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478449676 478466694 478466704 478448892 478456...,478466694
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478449676 478466694 478466704 478448892 478456...,478466704
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478449676 478466694 478466704 478448892 478456...,478448892
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478449676 478466694 478466704 478448892 478456...,478456792


In [44]:
#중복된 컬럼 삭제
del data['link_id_x']
data.rename(columns={'link_id_y':'link_id'},inplace=True)
data

,정류장ID,정류장명,동,geometry,lon,lat,인구_계,link_id
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478449676
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478466694
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478466704
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478448892
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478456792
...,...,...,...,...,...,...,...,...
927,202000238,진흥아파트,화서2동,POINT (126.9850962 37.28950644),126.985096,37.289506,26776.0,478442390
927,202000238,진흥아파트,화서2동,POINT (126.9850962 37.28950644),126.985096,37.289506,26776.0,478446861
927,202000238,진흥아파트,화서2동,POINT (126.9850962 37.28950644),126.985096,37.289506,26776.0,478446254
927,202000238,진흥아파트,화서2동,POINT (126.9850962 37.28950644),126.985096,37.289506,26776.0,478444165


### 2-3 버스정류소 데이터에 혼잡빈도강도 추가하기위해 버스정류소의 link_id와 crowded 데이터의 link_id merge

In [45]:
data = pd.merge(data,crowded[['link_id','혼잡빈도강도','혼잡시간강도']],how='left',on=['link_id'])

In [46]:
data.head()

,정류장ID,정류장명,동,geometry,lon,lat,인구_계,link_id,혼잡빈도강도,혼잡시간강도
0,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478449676,337.38,385.36
1,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478466694,250.20,349.80
2,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478466704,74.92,169.32
3,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478448892,49.42,185.96
4,201000313,권선구청,서둔동,POINT (126.9709828 37.25698318),126.970983,37.256983,38211.0,478456792,384.52,397.98


In [47]:
#그룹화해서 평균을 구할  변수만 선택
crowded_mean =data[['정류장ID','혼잡빈도강도','혼잡시간강도']]
crowded_mean.head(10)

,정류장ID,혼잡빈도강도,혼잡시간강도
0,201000313,337.38,385.36
1,201000313,250.20,349.80
2,201000313,74.92,169.32
3,201000313,49.42,185.96
4,201000313,384.52,397.98
5,201000313,348.20,388.88
6,201000313,155.76,301.26
7,201000313,258.66,346.44
8,201000313,195.06,291.50
9,201000345,375.72,396.20


In [48]:
crowded_mean=crowded_mean.groupby(['정류장ID'], as_index=False).mean()
crowded_mean

,정류장ID,혼잡빈도강도,혼잡시간강도
0,200000005,165.936667,237.006667
1,200000006,220.786000,284.800000
2,200000007,167.468571,242.294286
3,200000008,NaN,NaN
4,200000009,64.350000,70.273333
...,...,...,...
923,203000424,227.720000,312.191667
924,203000438,282.558000,320.137000
925,203000439,288.614000,324.329000
926,203000449,231.132000,284.934000


In [49]:
# 평균내지않은 변수 중복 제거
#(위에서 도로 개수대로 행을 생성했기 때문에, 도로 link_id 열을 제거 후 중복제거하면 다시 원래 행의 길이로 돌아옴)
#이 데이터를 기준으로 혼잡빈도강도 열을 추가할 것
data = data[['정류장ID','정류장명','동','lon','lat','인구_계']]
data = data.drop_duplicates()
data

,정류장ID,정류장명,동,lon,lat,인구_계
0,201000313,권선구청,서둔동,126.970983,37.256983,38211.0
9,201000345,남수원초등학교,세류2동,127.016000,37.251091,23728.0
17,201000379,신미주아파트,금곡동,126.953845,37.268806,45794.0
25,201000055,농수산물시장,권선1동,127.029815,37.256755,24895.0
28,201000268,곡반중학교,곡선동,127.034254,37.246081,27497.0
...,...,...,...,...,...,...
8039,202000249,화서오거리,화서1동,126.999697,37.278865,23411.0
8043,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,화서2동,126.995823,37.284316,26776.0
8053,202000246,엘지아파트.금강아파트,화서2동,126.981714,37.287994,26776.0
8059,202000239,꽃뫼신동아아파트,화서2동,126.986076,37.286663,26776.0


In [50]:
#인덱스 리셋
data.reset_index(inplace=True,drop=True)
data

,정류장ID,정류장명,동,lon,lat,인구_계
0,201000313,권선구청,서둔동,126.970983,37.256983,38211.0
1,201000345,남수원초등학교,세류2동,127.016000,37.251091,23728.0
2,201000379,신미주아파트,금곡동,126.953845,37.268806,45794.0
3,201000055,농수산물시장,권선1동,127.029815,37.256755,24895.0
4,201000268,곡반중학교,곡선동,127.034254,37.246081,27497.0
...,...,...,...,...,...,...
923,202000249,화서오거리,화서1동,126.999697,37.278865,23411.0
924,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,화서2동,126.995823,37.284316,26776.0
925,202000246,엘지아파트.금강아파트,화서2동,126.981714,37.287994,26776.0
926,202000239,꽃뫼신동아아파트,화서2동,126.986076,37.286663,26776.0


In [51]:
#혼잡빈도강도 평균값을 가지는 열 추가
data_crowded_temporal = pd.merge(data,crowded_mean,how='left',on='정류장ID')
data_crowded_temporal = data_crowded_temporal[['정류장ID','혼잡빈도강도','혼잡시간강도']]
data_crowded_temporal

,정류장ID,혼잡빈도강도,혼잡시간강도
0,201000313,228.235556,312.944444
1,201000345,106.830000,157.968750
2,201000379,327.900000,377.447500
3,201000055,241.273333,342.006667
4,201000268,83.941250,149.207500
...,...,...,...
923,202000249,283.400000,343.120000
924,202000241,203.522000,290.126000
925,202000246,181.568333,244.003333
926,202000239,278.155385,336.763077


In [52]:
data_crowded_temporal.to_csv('data/data_crowded_temporal.csv',index=False)

In [53]:
data

,정류장ID,정류장명,동,lon,lat,인구_계
0,201000313,권선구청,서둔동,126.970983,37.256983,38211.0
1,201000345,남수원초등학교,세류2동,127.016000,37.251091,23728.0
2,201000379,신미주아파트,금곡동,126.953845,37.268806,45794.0
3,201000055,농수산물시장,권선1동,127.029815,37.256755,24895.0
4,201000268,곡반중학교,곡선동,127.034254,37.246081,27497.0
...,...,...,...,...,...,...
923,202000249,화서오거리,화서1동,126.999697,37.278865,23411.0
924,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,화서2동,126.995823,37.284316,26776.0
925,202000246,엘지아파트.금강아파트,화서2동,126.981714,37.287994,26776.0
926,202000239,꽃뫼신동아아파트,화서2동,126.986076,37.286663,26776.0


## 3. 교통량

Category1 에서 전처리한 교통량 데이터 사용하기

In [54]:
traffic = pd.read_csv('data/busTraffic_only_sum.csv',index_col='Unnamed: 0')
traffic

,정류장ID,정류장명,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량
0,200000005,광교산입구.경기대수원캠퍼스,138507.0,107567.0,698.0,30243.0
1,200000006,광교공원.경기대수원캠퍼스입구.연무시장,144981.0,114321.0,738.0,29921.0
2,200000007,광교산입구.경기대수원캠퍼스,136500.0,105852.0,693.0,29957.0
3,200000008,문암골,0.0,0.0,0.0,0.0
4,200000009,비석거리입구,538641.0,402702.0,14511.0,121426.0
...,...,...,...,...,...,...
923,203000424,수원시도시안전통합센터,486849.0,401579.0,8970.0,76298.0
924,203000438,광교카페거리입구,397516.0,327057.0,5089.0,65369.0
925,203000439,광교카페거리입구,395725.0,325472.0,5086.0,65166.0
926,203000449,대선초.e편한세상영통2차아파트,112658.0,90981.0,956.0,20718.0


In [55]:
del traffic['정류장명']

정류장별로 교통량을 구하기위해 정류장 id 기준으로 merge

In [56]:
data = pd.merge(data,traffic,how='left',on='정류장ID')

In [57]:
data.head()

,정류장ID,정류장명,동,lon,lat,인구_계,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량
0,201000313,권선구청,서둔동,126.970983,37.256983,38211.0,446123.0,355781.0,5243.0,85100.0
1,201000345,남수원초등학교,세류2동,127.016000,37.251091,23728.0,749320.0,612913.0,9680.0,126724.0
2,201000379,신미주아파트,금곡동,126.953845,37.268806,45794.0,131512.0,106414.0,592.0,24507.0
3,201000055,농수산물시장,권선1동,127.029815,37.256755,24895.0,495990.0,420745.0,4284.0,70964.0
4,201000268,곡반중학교,곡선동,127.034254,37.246081,27497.0,1031383.0,849483.0,15600.0,166306.0


## 4. 버스승하차

In [58]:
data2=pd.read_csv('2.수원시_버스정류장별_승하차이력(1).csv')
data3=pd.read_csv('3.수원시_버스정류장별_승하차이력(2).csv')
data4=pd.read_csv('4.수원시_버스정류장별_승하차이력(3).csv')
data5=pd.read_csv('5.수원시_버스정류장별_승하차이력(4).csv')
data6=pd.read_csv('6.수원시_버스정류장별_승하차이력(5).csv')

In [59]:
passenger=pd.concat([data2,data3,data4,data5,data6])
passenger

,일자,업체명,노선번호,노선유형,정류소명,정류소ID,전체 승차 건수,초승 건수,환승 건수,전체 하차 건수,하차 건수,미태그 건수
0,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,서부공영차고지,201000446,1,1,0,0,0,0
1,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,구운동주민센터.서수원도서관,201000173,4,4,0,0,0,0
2,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,구운중학교,201000171,4,4,0,0,0,0
3,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,웃거리,201000031,5,4,1,0,0,0
4,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,구운오거리.경기중소기업성장지원센터,201000030,5,5,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
360279,20201230,화성여객(주),H65,일반형시내버스,수영오거리.방송통신대입구,233000001,2,1,1,5,5,0
360280,20201230,화성여객(주),H65,일반형시내버스,한국방송통신대학교,201000429,0,0,0,1,1,0
360281,20201230,화성여객(주),H65,일반형시내버스,국립축산과학원,233000592,0,0,0,1,1,0
360282,20201231,화성여객(주),H65,일반형시내버스,수영오거리.방송통신대입구,233000001,2,1,1,7,7,0


In [60]:
#정류소별로 승하차건수 구하기(일자별로 되어있기때문에 sum)
pass_sum=passenger.groupby(['정류소ID'], as_index=False).sum()
pass_sum=pass_sum.loc[:,['정류소ID','전체 승차 건수','전체 하차 건수']]
pass_sum.rename(columns = {'정류소ID':'정류장ID'}, inplace = True)
pass_sum['버스승하차']=pass_sum['전체 승차 건수']+pass_sum['전체 하차 건수']
pass_sum

,정류장ID,전체 승차 건수,전체 하차 건수,버스승하차
0,200000001,533,3664,4197
1,200000002,978,18901,19879
2,200000003,2224,10218,12442
3,200000004,3221,14821,18042
4,200000005,9386,72860,82246
...,...,...,...,...
1109,203000449,15546,11746,27292
1110,203000450,169528,56309,225837
1111,228001883,56966,49233,106199
1112,233000001,46289,37921,84210


In [61]:
# 승하차건수를 버스정류소 데이터에 merge
data=pd.merge(data,pass_sum,how='left', on='정류장ID')
data

,정류장ID,정류장명,동,lon,lat,인구_계,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량,전체 승차 건수,전체 하차 건수,버스승하차
0,201000313,권선구청,서둔동,126.970983,37.256983,38211.0,446123.0,355781.0,5243.0,85100.0,15434.0,623.0,16057.0
1,201000345,남수원초등학교,세류2동,127.016000,37.251091,23728.0,749320.0,612913.0,9680.0,126724.0,5517.0,3474.0,8991.0
2,201000379,신미주아파트,금곡동,126.953845,37.268806,45794.0,131512.0,106414.0,592.0,24507.0,3018.0,26449.0,29467.0
3,201000055,농수산물시장,권선1동,127.029815,37.256755,24895.0,495990.0,420745.0,4284.0,70964.0,42604.0,73082.0,115686.0
4,201000268,곡반중학교,곡선동,127.034254,37.246081,27497.0,1031383.0,849483.0,15600.0,166306.0,142617.0,84751.0,227368.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,202000249,화서오거리,화서1동,126.999697,37.278865,23411.0,61673.0,46867.0,321.0,14485.0,20645.0,46788.0,67433.0
924,202000241,숙지공원.화서동굿모닝힐.화서다산도서관,화서2동,126.995823,37.284316,26776.0,130962.0,94439.0,658.0,35861.0,9481.0,46159.0,55640.0
925,202000246,엘지아파트.금강아파트,화서2동,126.981714,37.287994,26776.0,172323.0,124084.0,1302.0,46929.0,42040.0,99936.0,141976.0
926,202000239,꽃뫼신동아아파트,화서2동,126.986076,37.286663,26776.0,558920.0,441698.0,5998.0,111223.0,606.0,14692.0,15298.0


In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 928 entries, 0 to 927
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   정류장ID      928 non-null    int64  
 1   정류장명       928 non-null    object 
 2   동          928 non-null    object 
 3   lon        928 non-null    float64
 4   lat        928 non-null    float64
 5   인구_계       901 non-null    float64
 6   전체_추정교통량   928 non-null    float64
 7   승용차_추정교통량  928 non-null    float64
 8   버스_추정교통량   928 non-null    float64
 9   화물차_추정교통량  928 non-null    float64
 10  전체 승차 건수   919 non-null    float64
 11  전체 하차 건수   919 non-null    float64
 12  버스승하차      919 non-null    float64
dtypes: float64(10), int64(1), object(2)
memory usage: 101.5+ KB


## 5. 유동인구

In [63]:
float_popul = pd.read_csv('16.수원시_요일별_유동인구(2020).csv')
float_popul

,STD_YM,FLOW_POP_CNT_MON,FLOW_POP_CNT_TUS,FLOW_POP_CNT_WED,FLOW_POP_CNT_THU,FLOW_POP_CNT_FRI,FLOW_POP_CNT_SAT,FLOW_POP_CNT_SUN,lon,lat
0,202001,0.04,0.04,0.02,0.04,0.02,0.00,0.00,126.928999,37.284119
1,202001,0.26,0.25,0.26,0.25,0.26,0.34,0.34,126.929668,37.270151
2,202001,0.76,0.76,0.88,0.74,0.85,1.34,1.32,126.930229,37.270604
3,202001,1.78,1.78,1.78,1.77,1.84,1.71,1.71,126.930188,37.276012
4,202001,0.05,0.05,0.04,0.05,0.04,0.00,0.00,126.930674,37.286381
...,...,...,...,...,...,...,...,...,...,...
430248,202012,7.45,8.62,7.10,7.72,7.24,5.92,5.51,127.089134,37.294248
430249,202012,29.42,35.76,34.13,35.76,28.50,13.28,11.34,127.089129,37.295149
430250,202012,12.33,15.59,16.97,17.73,11.80,5.67,4.18,127.089126,37.295600
430251,202012,1.20,1.25,1.24,1.31,1.18,0.67,0.51,127.089116,37.297403


In [64]:
float_popul['week_sum'] = float_popul.apply(lambda x: x.FLOW_POP_CNT_MON + x.FLOW_POP_CNT_TUS+x.FLOW_POP_CNT_WED+x.FLOW_POP_CNT_THU+x.FLOW_POP_CNT_FRI, axis='columns')

In [65]:
float_popul['weekend_sum']=float_popul.apply(lambda x: x.FLOW_POP_CNT_SAT+x.FLOW_POP_CNT_SUN, axis='columns')


In [66]:
float_popul = float_popul[['STD_YM','week_sum','weekend_sum','lon','lat']]
float_popul

,STD_YM,week_sum,weekend_sum,lon,lat
0,202001,0.16,0.00,126.928999,37.284119
1,202001,1.28,0.68,126.929668,37.270151
2,202001,3.99,2.66,126.930229,37.270604
3,202001,8.95,3.42,126.930188,37.276012
4,202001,0.23,0.00,126.930674,37.286381
...,...,...,...,...,...
430248,202012,38.13,11.43,127.089134,37.294248
430249,202012,163.57,24.62,127.089129,37.295149
430250,202012,74.42,9.85,127.089126,37.295600
430251,202012,6.18,1.18,127.089116,37.297403


In [67]:
float_group = float_popul[['lon','lat','week_sum','weekend_sum']].groupby(['lon','lat'], as_index=False).sum()
float_group

,lon,lat,week_sum,weekend_sum
0,126.927348,37.278703,5.34,0.00
1,126.927389,37.273295,172.27,61.51
2,126.927392,37.272844,101.76,38.04
3,126.927395,37.272393,58.26,23.84
4,126.927399,37.271943,100.85,50.52
...,...,...,...,...
39410,127.089116,37.297403,23.93,5.23
39411,127.089126,37.295600,265.28,43.50
39412,127.089129,37.295149,2797.86,519.11
39413,127.089134,37.294248,310.76,101.40


In [68]:
# 좌표간 25m 이내 ->버스정류장이 해당 격자에 포함됨

In [69]:
bus_array

array([['201000313', 126.9709828, 37.25698318],
       ['201000345', 127.0160003, 37.25109066],
       ['201000379', 126.9538449, 37.26880555],
       ...,
       ['202000246', 126.98171380000001, 37.28799446],
       ['202000239', 126.98607630000001, 37.28666265],
       ['202000238', 126.9850962, 37.28950644]], dtype=object)

In [70]:
#유동인구 좌표 array생성
#도로 좌표 array로 만들기
float_array = float_group[['week_sum','weekend_sum','lon','lat']]
float_array = float_array.to_numpy()
float_array

array([[   5.34      ,    0.        ,  126.92734768,   37.27870277],
       [ 172.27      ,   61.51      ,  126.92738865,   37.27329463],
       [ 101.76      ,   38.04      ,  126.92739206,   37.27284396],
       ...,
       [2797.86      ,  519.11      ,  127.0891287 ,   37.29514923],
       [ 310.76      ,  101.4       ,  127.0891336 ,   37.29424785],
       [  42.06      ,   35.79      ,  127.08913605,   37.29379717]])

In [71]:
#반경 25m 이내인 도로의 유동인구를 data에 추가
data['평일유동인구']=''
data['주말유동인구']=''
for i in range(bus_array.shape[0]):
    for j in range(float_array.shape[0]):
        h_dis = GeoUtil.get_harversion_distance(bus_array[i,1], bus_array[i,2], float_array[j,2],float_array[j,3])
        if h_dis<=0.025:  
            data['평일유동인구'][i] = float(float_array[j,0])
            data['주말유동인구'][i] = float(float_array[j,1])
    

    if type(data['평일유동인구'][i])!=float:
        data['평일유동인구'][i] = np.nan
        data['주말유동인구'][i] = np.nan

In [72]:
data.head()

,정류장ID,정류장명,동,lon,lat,인구_계,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량,전체 승차 건수,전체 하차 건수,버스승하차,평일유동인구,주말유동인구
0,201000313,권선구청,서둔동,126.970983,37.256983,38211.0,446123.0,355781.0,5243.0,85100.0,15434.0,623.0,16057.0,12430.9,2878.7
1,201000345,남수원초등학교,세류2동,127.016000,37.251091,23728.0,749320.0,612913.0,9680.0,126724.0,5517.0,3474.0,8991.0,14741.4,4190.02
2,201000379,신미주아파트,금곡동,126.953845,37.268806,45794.0,131512.0,106414.0,592.0,24507.0,3018.0,26449.0,29467.0,11751.9,5055.53
3,201000055,농수산물시장,권선1동,127.029815,37.256755,24895.0,495990.0,420745.0,4284.0,70964.0,42604.0,73082.0,115686.0,NaN,NaN
4,201000268,곡반중학교,곡선동,127.034254,37.246081,27497.0,1031383.0,849483.0,15600.0,166306.0,142617.0,84751.0,227368.0,33588.2,9487.21


In [73]:
data['평일유동인구'].isnull().sum()

204

In [74]:
# 유동인구 좌표 -> 50m x 50 격자로 되어있음
# 위에서 유동인구 좌표기준  반지름 25m의 원에 속하는 좌표를 찾아냈기때문에
# 50mX50m - (반지름 25m의 원)에 해당하는 구간에 포함되는 좌표 구하기
from math import isnan
for i in range(bus_array.shape[0]):
    min_dis = np.inf
    min_index = 0
    if isnan(data['평일유동인구'][i]):
        for j in range(float_array.shape[0]):
            h_dis = GeoUtil.get_harversion_distance(bus_array[i,1], bus_array[i,2], float_array[j,2],float_array[j,3])
            if h_dis>0.025 and h_dis<((0.05**2+0.05**2)**0.5)/2:
                if h_dis<min_dis:
                    min_dis = h_dis
                    min_index = j
                    data['평일유동인구'][i] = float(float_array[min_index,0])
                    data['주말유동인구'][i] = float(float_array[min_index,1])

In [75]:
data.head(10)

,정류장ID,정류장명,동,lon,lat,인구_계,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량,전체 승차 건수,전체 하차 건수,버스승하차,평일유동인구,주말유동인구
0,201000313,권선구청,서둔동,126.970983,37.256983,38211.0,446123.0,355781.0,5243.0,85100.0,15434.0,623.0,16057.0,12430.9,2878.7
1,201000345,남수원초등학교,세류2동,127.016000,37.251091,23728.0,749320.0,612913.0,9680.0,126724.0,5517.0,3474.0,8991.0,14741.4,4190.02
2,201000379,신미주아파트,금곡동,126.953845,37.268806,45794.0,131512.0,106414.0,592.0,24507.0,3018.0,26449.0,29467.0,11751.9,5055.53
3,201000055,농수산물시장,권선1동,127.029815,37.256755,24895.0,495990.0,420745.0,4284.0,70964.0,42604.0,73082.0,115686.0,11958.1,3870.45
4,201000268,곡반중학교,곡선동,127.034254,37.246081,27497.0,1031383.0,849483.0,15600.0,166306.0,142617.0,84751.0,227368.0,33588.2,9487.21
5,201000337,경기도건설본부.경기도동물위생시험소.칠보마을7단지,금곡동,126.941563,37.267677,45794.0,128507.0,104425.0,828.0,23252.0,55552.0,24371.0,79923.0,5163.6,1859.03
6,200000292,백선교회,파장동,126.977691,37.311610,23573.0,112707.0,93269.0,1128.0,18310.0,499.0,14125.0,14624.0,2500.41,1044.14
7,202000089,화서역,화서2동,126.990494,37.284140,26776.0,387598.0,315910.0,4646.0,67047.0,77489.0,42006.0,119495.0,16861.5,6263.79
8,201000358,한양수자인파크원아파트.금호초등학교,금곡동,126.952716,37.265969,45794.0,275543.0,225131.0,2160.0,48252.0,21870.0,92639.0,114509.0,15894.6,5222.96
9,201000357,수원은혜교회.한양수자인파크원아파트,금곡동,126.953978,37.266016,45794.0,227483.0,186718.0,1900.0,38864.0,84062.0,24182.0,108244.0,17893.4,6906.82


In [76]:
data['평일유동인구'].isnull().sum()

0

In [77]:
data.head()

,정류장ID,정류장명,동,lon,lat,인구_계,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량,전체 승차 건수,전체 하차 건수,버스승하차,평일유동인구,주말유동인구
0,201000313,권선구청,서둔동,126.970983,37.256983,38211.0,446123.0,355781.0,5243.0,85100.0,15434.0,623.0,16057.0,12430.9,2878.7
1,201000345,남수원초등학교,세류2동,127.016000,37.251091,23728.0,749320.0,612913.0,9680.0,126724.0,5517.0,3474.0,8991.0,14741.4,4190.02
2,201000379,신미주아파트,금곡동,126.953845,37.268806,45794.0,131512.0,106414.0,592.0,24507.0,3018.0,26449.0,29467.0,11751.9,5055.53
3,201000055,농수산물시장,권선1동,127.029815,37.256755,24895.0,495990.0,420745.0,4284.0,70964.0,42604.0,73082.0,115686.0,11958.1,3870.45
4,201000268,곡반중학교,곡선동,127.034254,37.246081,27497.0,1031383.0,849483.0,15600.0,166306.0,142617.0,84751.0,227368.0,33588.2,9487.21


위에서전처리한 혼잡빈도강도와 결합

In [78]:
data_crowded_temporal

,정류장ID,혼잡빈도강도,혼잡시간강도
0,201000313,228.235556,312.944444
1,201000345,106.830000,157.968750
2,201000379,327.900000,377.447500
3,201000055,241.273333,342.006667
4,201000268,83.941250,149.207500
...,...,...,...
923,202000249,283.400000,343.120000
924,202000241,203.522000,290.126000
925,202000246,181.568333,244.003333
926,202000239,278.155385,336.763077


In [79]:
#혼잡도 데이터 결합
data = pd.merge(data,data_crowded_temporal,how='left',on='정류장ID')

In [80]:
data.head()

,정류장ID,정류장명,동,lon,lat,인구_계,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량,전체 승차 건수,전체 하차 건수,버스승하차,평일유동인구,주말유동인구,혼잡빈도강도,혼잡시간강도
0,201000313,권선구청,서둔동,126.970983,37.256983,38211.0,446123.0,355781.0,5243.0,85100.0,15434.0,623.0,16057.0,12430.9,2878.7,228.235556,312.944444
1,201000345,남수원초등학교,세류2동,127.016000,37.251091,23728.0,749320.0,612913.0,9680.0,126724.0,5517.0,3474.0,8991.0,14741.4,4190.02,106.830000,157.968750
2,201000379,신미주아파트,금곡동,126.953845,37.268806,45794.0,131512.0,106414.0,592.0,24507.0,3018.0,26449.0,29467.0,11751.9,5055.53,327.900000,377.447500
3,201000055,농수산물시장,권선1동,127.029815,37.256755,24895.0,495990.0,420745.0,4284.0,70964.0,42604.0,73082.0,115686.0,11958.1,3870.45,241.273333,342.006667
4,201000268,곡반중학교,곡선동,127.034254,37.246081,27497.0,1031383.0,849483.0,15600.0,166306.0,142617.0,84751.0,227368.0,33588.2,9487.21,83.941250,149.207500


In [81]:
data['평일유동인구'] = data['평일유동인구'].astype(float)
data['주말유동인구'] = data['주말유동인구'].astype(float)

In [82]:
data[['인구_계','전체_추정교통량','버스_추정교통량','버스승하차','평일유동인구','주말유동인구','혼잡빈도강도','혼잡시간강도']].corr()

,인구_계,전체_추정교통량,버스_추정교통량,버스승하차,평일유동인구,주말유동인구,혼잡빈도강도,혼잡시간강도
인구_계,1.000000,-0.109126,-0.067993,-0.202295,0.013615,0.010698,-0.015201,0.037431
전체_추정교통량,-0.109126,1.000000,0.943637,0.147830,0.234678,0.234867,-0.092434,-0.188906
버스_추정교통량,-0.067993,0.943637,1.000000,0.078984,0.210287,0.217347,-0.069214,-0.163446
버스승하차,-0.202295,0.147830,0.078984,1.000000,0.269604,0.256832,0.086502,-0.002484
평일유동인구,0.013615,0.234678,0.210287,0.269604,1.000000,0.952471,-0.064602,-0.118700
주말유동인구,0.010698,0.234867,0.217347,0.256832,0.952471,1.000000,-0.073002,-0.126586
혼잡빈도강도,-0.015201,-0.092434,-0.069214,0.086502,-0.064602,-0.073002,1.000000,0.923802
혼잡시간강도,0.037431,-0.188906,-0.163446,-0.002484,-0.118700,-0.126586,0.923802,1.000000


In [83]:
data.to_csv('data/category2_after_preprocessing.csv',index=False)